# Урок 4. Машинное обучение

Конечно, для Julia существует имплементация всеми любимого [ScikitLearn](https://github.com/cstjean/ScikitLearn.jl/). Прежде чем мы начнем, конечно нам нужно раздобыть данные, но чтобы далеко не ходить, мы просто возьмем данные из [Титаника](https://www.kaggle.com/c/titanic/data) и совсем чуть-чуть их подготовим

In [ ]:
#import Pkg;
#Pkg.add("HTTP")
#Pkg.add("ScikitLearn")
#Pkg.add("CategoricalArrays")
#Pkg.add("Revise")
#Pkg.add("DecisionTree")

In [ ]:
using CSV
using HTTP # Модуль для работы с HTTP
using DataFrames
using ScikitLearn
using CategoricalArrays # Модуль для работы с Категориальными переменными

## 0. Подготовка данных

In [ ]:
url_train = "https://raw.githubusercontent.com/JuliaEvangelists/Julia-in-DS/main/data/titanic/train.csv"
url_test = "https://raw.githubusercontent.com/JuliaEvangelists/Julia-in-DS/main/data/titanic/test.csv"

Кстати, мы не рассказали Вам в уроке 2, но модуль [CSV](https://juliadata.github.io/CSV.jl/stable/) может использовать модуль [HTTP](https://juliaweb.github.io/HTTP.jl/stable/) для того чтобы взять данные которые доступны в интернете:

In [ ]:
train = CSV.read(HTTP.get(url_train).body)
test = CSV.read(HTTP.get(url_test).body)

size(train), size(test)

In [ ]:
first(train, 5)

Сделаем небольшие предобработки и для начала заменим пустые значения нулями, для этого воспользуемся методом `coalesce.`, подробнее о работе с пропусками можно ознакомиться [тут](http://juliadata.github.io/DataFrames.jl/stable/man/missing/)

In [ ]:
train_ready = coalesce.(train, 0) 
test_ready = coalesce.(test, 0) 
@show 

Удалим бесполезные столбцы:

In [ ]:
train_ready = select(train_ready, Not([:Name, :PassengerId, :Ticket, :Cabin]))
test_ready = select(test_ready, Not([:Name, :PassengerId, :Ticket, :Cabin]))

@show

Затем, мы избавимся от текстовых данных. Еще один способ как работать с категориальными переменными Вы можете найти [тут](http://juliadata.github.io/DataFrames.jl/stable/man/categorical/)

In [ ]:
unique(train_ready.Embarked), unique(train_ready.Sex)

In [ ]:
train_ready.Sex = recode(train_ready.Sex, "male" => 1, "female" => 0)
test_ready.Sex = recode(test_ready.Sex, "male" => 1, "female" => 0)
@show

Сделаем One Hot Encoding

In [ ]:
train_ready.Embarked_S = recode(train_ready.Embarked, 0, "S" => 1)
train_ready.Embarked_C = recode(train_ready.Embarked, 0, "C" => 1)
train_ready.Embarked_Q = recode(train_ready.Embarked, 0, "Q" => 1)

test_ready.Embarked_S = recode(test_ready.Embarked, 0, "S" => 1)
test_ready.Embarked_C = recode(test_ready.Embarked, 0, "C" => 1)
test_ready.Embarked_Q = recode(test_ready.Embarked, 0, "Q" => 1)


train_ready = select(train_ready, Not([:Embarked]))
test_ready = select(test_ready, Not([:Embarked]))

first(train_ready, 3)

Сформируем финальные датасеты:

In [ ]:
X = convert(Array{Float64,2}, select(train_ready, Not("Survived")))
y = reshape(convert(Array, select(train_ready, "Survived")),  (891))

test = convert(Array{Float64,2}, test_ready)
@show

## 1. Пора учить модельки


Реализация [ScikitLearn](https://github.com/cstjean/ScikitLearn.jl/) в Julia поддерживает все классы из [оригинальной Python](https://scikit-learn.org/stable/index.html) библиотеки. 

Чтобы их импортировать можно использовать декоратор `@sk_import`, для демонстрации импортируем логистическую регрессию:

In [ ]:
@sk_import linear_model: LogisticRegression

Каждая такая модель поддерживает все гиперпараметры для настройки, давайте посмотрим их для логистической регрессии: 

In [ ]:
?LogisticRegression

Создадим первую модель

In [ ]:
model = LogisticRegression(fit_intercept=true, max_iter = 200)

Как видите, она является `PyObject` и теперь мы можем:
- обучить ее

In [ ]:
fit!(model, X, y);

- сделать предсказания, например на трейне и посчитать любую метрику доступную в ScikitLearn: 

In [ ]:
@sk_import metrics: accuracy_score
preds = predict(model, X)
accuracy_score(preds, y)

Если Вы заметили, то мы использовали метод `predict`, но если вдруг потребуется сделать апостериорную оценку вероятности отнесения объекта к класу, то нужно воспользоваться методом `predict_proba`

In [ ]:
preds_prob = DataFrame(predict_proba(model, X))
first(preds_prob, 5)

Чтобы достать коэффициенты из обученной модели, необходимо обратиться к методу `.coef_`

In [ ]:
model.coef_

Больше методов ищите в [документации](https://cstjean.github.io/ScikitLearn.jl/dev/man/api/) 

## 2. Кросс валидация 
Валидация - залог успеха реализация в Julia позволяет это сделать без СМС и регистрации

In [ ]:
scores = CrossValidation.cross_val_score(model, X, y; cv=5)

In [ ]:
using Statistics

scores_std = std(scores)
scores_mean = mean(scores)

print("Accuracy: $scores_mean (+/- $scores_std)")

Кроме того, мы можем использовать любые стратегии для разбиения на фолды

In [ ]:
using ScikitLearn.CrossValidation: KFold
using ScikitLearn.CrossValidation: cross_val_score

In [ ]:
scores = cross_val_score(model, X, y; cv=KFold(size(X)[1], n_folds=10))

## 3. Pipelines

Настоящему Data Science специалисту будет явно недостаточно сделать просто `fit` и `predict`, ему захочется строить сложные мультистейджинговые модели, попробуем в качестве примера построить пайплайн:  
- MinMaxScaler $\longrightarrow$ PCA $\longrightarrow$ LogisticRegression

In [ ]:
using ScikitLearn.Pipelines: Pipeline
@sk_import preprocessing: MinMaxScaler
@sk_import decomposition: PCA

Создадим объекты пайплайна

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
reducer = PCA(n_components=5)
model_l1 = LogisticRegression(fit_intercept=true, max_iter = 200, penalty="l1", solver="liblinear")
@show

Соберем весь пайплайн

In [ ]:
estimators = [("min_max_scaler", scaler), ("reduce_dim", reducer), ("logistic_regression", model_l1)]

pipe = Pipeline(estimators)

Проведем кросс валидацию на 5 фолдах, чтобы проверить модель

In [ ]:
scores = cross_val_score(pipe, X, y; cv=5)
scores_std = std(scores)
scores_mean = mean(scores)
print("Accuracy: $scores_mean (+/- $scores_std)")

Если нас устроило качество, то обучим модель и сделаем предсказания для тестовой выборки

In [ ]:
fit!(pipe, X, y)
predictions = predict_proba(pipe, test)

predictions[1:4,:]

Пришло время отправить наше решение на [kaggle](https://www.kaggle.com/c/titanic/data)

In [ ]:
url_submission = "https://raw.githubusercontent.com/JuliaEvangelists/Julia-in-DS/main/data/titanic/gender_submission.csv"
sample_submission = CSV.read(HTTP.get(url_submission).body)

first(sample_submission, 3)
sample_submission.Survived = predictions[:, 1]

CSV.write("submission.csv", sample_submission)

Самостоятельно поэкспериментирйте с настройкой параметров с помощью [Grid Search](https://cstjean.github.io/ScikitLearn.jl/dev/man/model_selection/)

## 4. Нативные Julia модели

Модели написанные на Julia размещаются в других пакетах и их необходимо установить отдельно с помощью `Pkg.add`. Среди таких моделей есть например реализации:
- [GaussianMixtures](https://github.com/davidavdav/GaussianMixtures.jl)
- [DecisionTree](https://github.com/bensadeghi/DecisionTree.jl)

Воспользуемся последней:

In [ ]:
using DecisionTree: DecisionTreeClassifier, print_tree

Обучим модель и сделаем предсказание

In [ ]:
model = DecisionTreeClassifier(max_depth=5)
fit!(model, X, y)
# apply learned model
predict(model, test)[1:5, :]
# get the probability of each label

Кстати, в этом пакете есть удобное средство для визуализации построенного дерева

In [ ]:
print_tree(model, 5)

## 5. Сравним модели Julia и Python
  
Любознательному читателю было бы интересно сравнить скорость обучения для разных реализаций. Сделаем это для RandomForestClassifier!  

In [ ]:
import DecisionTree
@sk_import ensemble: RandomForestClassifier

model_py = RandomForestClassifier(n_estimators=1000)
model_native = DecisionTree.RandomForestClassifier(n_trees=1000)

In [ ]:
@time score_native =  cross_val_score(model_native, X, y, cv=5)
@time score_py = cross_val_score(model_py, X, y, cv=5)

score_native = [mean(score_native)-std(score_native), mean(score_native)+std(score_native)]
score_py = [mean(score_py)-std(score_py), mean(score_py)+std(score_py)]

println()
println("Среднее качество для кросс валидации на 5 фолдах:")
println("для Julia: $score_native")
println("для Python: $score_py")

## 6. Другие интересные пакеты 

- [MLJ](https://github.com/alan-turing-institute/MLJ.jl) от Института Алана Тьюринга - это набор инструментов, написанный на языке Julia, предоставляющий общий интерфейс и мета-алгоритмы для выбора, настройки, оценки, составления и сравнения моделей машинного обучения, написанных на языке Julia. В частности, MLJ оборачивает большое количество моделей scikit-learn.     
- [CombineML](https://github.com/ppalmes/CombineML.jl) используется для обучения различных ансамблей над алгоритмами.
- [AutoMLPipeline](https://github.com/IBM/AutoMLPipeline.jl) для упрощения построения пайплайнов
- [TSML](https://github.com/IBM/TSML.jl) для работы с временными рядами
- [Flux](https://fluxml.ai/Flux.jl/stable/) позволяет учить нейронные сети и другие алгоритмы легко и непринужденно подключая GPU. Этот пакет мы рассмотрим уже в следующем уроке. 